## Библиотеки

In [57]:
import matplotlib.pyplot as plt
from matplotlib.image import imread
from mpl_toolkits import mplot3d
from copy import deepcopy
from mlxtend.plotting import plot_decision_regions
import seaborn as sns
import pandas as pd
from tqdm.notebook import tqdm
from scipy.spatial.distance import cdist
import numpy as np
from sklearn import tree, base
import itertools
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier, 
                              GradientBoostingClassifier, BaggingClassifier)
from sklearn.svm import SVC, SVR
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import classification_report, roc_auc_score, roc_curve
from sklearn.model_selection import KFold, ParameterGrid
from sklearn.model_selection import cross_val_score, RepeatedStratifiedKFold, RepeatedKFold
from sklearn.datasets import make_classification, make_regression, load_wine, load_boston
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error, r2_score
from torchvision import datasets, transforms
import torch

import xgboost as xgb
from catboost import CatBoostClassifier
import lightgbm as lgb

# Домашнее задание

**Objective**: Реализовать комитетный бустинг для задачи регрессии.

Фактически, будет тот же код, что и для GBR из семинара, однако теперь мы будем сортировать объекты по величине отклонения и брать те, что находятся в середине.



In [ ]:
class ComBoostRegressor(object):
    def __init__(self, base_estimator=None, n_estimators=10):
        self.n_estimators = n_estimators
        self.base_estimator = DecisionTreeRegressor(max_depth=1)
        if base_estimator:
            self.base_estimator = base_estimator
            
        self.b = [base.clone(self.base_estimator) for _ in range(self.n_estimators)]
        
    def get_params(self, deep=True):
        return {'n_estimators': self.n_estimators, 
                'base_estimator': self.base_estimator}
    
    def score(self, X, Y):
        return r2_score(Y, self.predict(X))
    
    def predict(self, X):
        return np.sum([elem.predict(X) for elem in self.b], axis=0)
    
    def fit(self, X, Y, l0=0, l1=100, l2=None, dl=100):
        if l2 is None:
            l2 = len(X)
        
        # learn t-th model to predict difference btw predict and target
        residual = Y.copy()
        for t, b in enumerate(self.b):
            if t == 0:
                b.fit(X, residual)
                residual = Y  - b.predict(X)
            else:
                sorted_ids = np.argsort(residual ** 2)
                # sort objects with respect to value of residual
                X_sorted = X[sorted_ids]
                residual_sorted = residual[sorted_ids]
                
                dict_of_param = []
                
                for k in range(l1, l2, dl):
                    new_item = {'l0': l0, 'k': k}
                    
                    # fit local t-th estimator
                    local_b = base.clone(self.base_estimator)
                    local_b.fit(X_sorted[l0:k], residual_sorted[l0:k])
                    
                    # update res
                    local_pred = local_b.predict(X)
                    new_res = residual_sorted - local_pred
                    
                    new_item['Q'] = np.mean(new_res ** 2)
                    dict_of_param.append(new_item)
                
                # take object btw best boundaries
                element = sorted(dict_of_param, key=lambda x: x['Q'])[0]
                b.fit(X_sorted[element['l0']: element['k']],
                      residual_sorted[element['l0']: element['k']])
                
                # update residual
                residual = residual - b.predict(X)
                

Сгенерируем выборку:

In [ ]:
X, y = make_regression(n_samples=1000, n_features=20, random_state=6)

Рассмотрим наш алгоритм в сравнении с различными базовыми алгоритамами.

## DecisionTree

In [95]:
X, y = make_regression(n_samples=1000, n_features=20, random_state=6)
model = DecisionTreeRegressor(max_depth=3)

cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(model, X, y, cv=cv, n_jobs=-1)
print('SCORE: %.2f (%.2f)' % (np.mean(n_scores), np.std(n_scores)))

SCORE: 0.46 (0.07)


В ходе экспрериментов было установлено, что, как ни странно, лучше всего себя показывают решающие пни, т.е. деревья с глубиной 1:

In [96]:
X, y = make_regression(n_samples=1000, n_features=20, random_state=6)
model = ComBoostRegressor(DecisionTreeRegressor(max_depth=1), n_estimators=1000)

n_scores = cross_val_score(model, X, y, cv=cv, n_jobs=-1)
print('SCORE: %.2f (%.2f)' % (np.mean(n_scores), np.std(n_scores)))

SCORE: 0.90 (0.06)


Для сравнения, качество дерева с глубиной 20:

In [104]:
model = DecisionTreeRegressor(max_depth=20)

n_scores = cross_val_score(model, X, y, cv=cv, n_jobs=-1)
print('SCORE: %.2f (%.2f)' % (np.mean(n_scores), np.std(n_scores)))

SCORE: 0.57 (0.07)


## SVM

In [99]:
model = SVR(kernel='rbf')

n_scores = cross_val_score(model, X, y, cv=cv, n_jobs=-1)
print('SCORE: %.2f (%.2f)' % (np.mean(n_scores), np.std(n_scores)))

SCORE: 0.11 (0.01)


In [105]:
model = ComBoostRegressor(SVR(kernel='rbf'), n_estimators=1000)

n_scores = cross_val_score(model, X, y, cv=cv, n_jobs=-1)
print('SCORE: %.2f (%.2f)' % (np.mean(n_scores), np.std(n_scores)))

SCORE: 0.50 (0.15)


## Linear regression

In [114]:
model = LinearRegression()

n_scores = cross_val_score(model, X, y, cv=cv, n_jobs=-1)
print('SCORE: %.2f (%.2f)' % (np.mean(n_scores), np.std(n_scores)))

SCORE: 1.00 (0.00)


In [115]:
model = ComBoostRegressor(LinearRegression(), n_estimators=10)

n_scores = cross_val_score(model, X, y, cv=cv, n_jobs=-1)
print('SCORE: %.2f (%.2f)' % (np.mean(n_scores), np.std(n_scores)))

SCORE: 1.00 (0.00)


## Выводы

1. Алгоритм работает, так как почти во всех случаях удалось добиться ощутимой прибавки к качеству.
2. Неожиданно хорошо себя показали деревья глубины 1, обеспечив очень высокий показатель R_2 при приемлемой погрешности.
3. Сложные алгоритмы вроде SVM показывают себя при использовании в бустинге ожидаемо плохо. Справедливости ради, стоит отметить, что качество SVM на данной выборке также мало по сравнению даже с простыми деревьями.
4. В целом, полученные результаты позволяют сделать вывод об адекватности алгоритма ComBoost.